In [1]:
import numpy as np
import pandas as pd

import scipy.io
from scipy.io import loadmat
import matplotlib.pyplot as plt
from matplotlib import cm
import cmocean

import xarray as xr


# Curl Plot
OG code by: Sawyer Brand

In [4]:
def curl(csig,lonmax,lonmin,latmax,latmin,floats,stations):
    Floats = np.loadtxt(floats) #here, I have created a new variable that loads the float txt file when used

    Lonny = Floats[:,2] #here, I have created a new variable that is made up of all the rows (:) in the third column (2)
    # of the float text file. The [:,2] means the indices that correspond will all the rows of the third column.
    Latty = Floats[:,1] #here, I have created a new variable that is made up of all the rows (:) in the second column (1)
    # of the float text file. The [:,2] means the indices that correspond will all the rows of the third column.

    curl = scipy.io.loadmat('CurlFixed.mat')
    vec = scipy.io.loadmat('VectorFixed.mat')
    
    Curly=np.asarray(curl['Image'][:,:])
    lon=np.asarray(curl['Lon'][:,0])
    lat=np.asarray(curl['Lat'][:,0])
    Curly.shape

    [xx,yy]=np.meshgrid(lon,lat)

    ang = np.asarray(vec['ang'][:,:])
    speed = np.asarray(vec['speed'][:,:])
    lon2 = np.asarray(vec['lon'][:,0])
    lat2 = np.asarray(vec['lat'][:,0])

    [XX,YY]=np.meshgrid(lon2,lat2)

    #So this is where the boundaries that you set up above come into play. You want to zoom into the area the cruise
    # is taking place but you don't want to just set the boundaries of the plot because it will still be processing 
    # the entire dataset. So instead, you'll limit the dataset itself to the area you want

    Curl = Curly[(xx>=LonMin)&(xx<=LonMax)&(yy>=LatMin)&(yy<=LatMax)]
    Lon = lon[(lon>=LonMin)&(lon<=LonMax)]
    Lat = lat[(lat>=LatMin)&(lat<=LatMax)]

    Curl = Curl.reshape(len(Lat),len(Lon))
    #In the above code, you've selected for the indices (which are the numbers that determine the location of the numbers
    # within a matrix or vector or whatever) that correspond to the longitude and latitude set you want. 
    
    Ang = ang[(XX>=LonMin)&(XX<=LonMax)&(YY>=LatMin)&(YY<=LatMax)]
    Speed = speed[(XX>=LonMin)&(XX<=LonMax)&(YY>=LatMin)&(YY<=LatMax)]
    Lon2 = lon2[(lon2>=LonMin)&(lon2<=LonMax)]
    Lat2 = lat2[(lat2>=LatMin)&(lat2<=LatMax)]

    Ang = Ang.reshape(len(Lat2),len(Lon2))*0.0174533
    Speed = Speed.reshape(len(Lat2),len(Lon2))

    U = np.array(Speed*np.cos(Ang))
    V = np.array(Speed*np.sin(Ang))
    
    
    plt.figure(1,(30,20))
    ax = plt.axes(projection=ccrs.PlateCarree())
    
    plt.plot(Lonny,Latty,'k-')

    im = plt.contourf(Lon,Lat,Curl,1000)
    plt.contour(Lon,Lat,Curl,70,cmap='gist_gray')
    plt.quiver(Lon2,Lat2,U,V)
    cbar = plt.colorbar(im,fraction=0.05)
    
    plt.plot(Lonny,Latty,'r.',markersize=30,markeredgecolor='green');
    
    ax.add_feature(cfeature.LAND,facecolor='black')
    
    plt.title('NCEP Wind Stress Curl for '+csig,fontsize=30)
    
    plt.xlim([LonMin,LonMax])
    plt.ylim([LatMin,LatMax])
    
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')
    
    gl.xlabels_top = False
    gl.ylabels_left = False
    gl.xlines = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 20, 'color': 'black'}
    gl.ylabel_style = {'size': 20, 'color': 'black'}

    plt.legend()
    
    plt.savefig(csig+'/'+csig+'_Curl.png')
    Image.open(csig+'/'+csig+'_Curl.png').save(csig+'/'+csig+'_Curl.jpg','JPEG')